In [1]:
import json
import sys
import os
sys.path.append('/Users/gisaac/Documents/GitHub/river_flow_notebooks/')
with open("/Users/gisaac/Documents/GitHub/river_flow_notebooks/01104200.json") as f:
    data = json.load(f)

In [2]:
from scraping_functions import HydroScraper

In [3]:
from datetime import datetime
h = HydroScraper(datetime(2023, 1, 1), datetime(2024, 1, 30), "/Users/gisaac/Documents/GitHub/river_flow_notebooks/01104200.json", "weather_asos_final")

Getting request from USGS
http://waterservices.usgs.gov/nwis/iv/?format=rdb,1.0&sites=01104200&startDT=2023-01-01&endDT=2024-01-30&parameterCd=00060,00065,00045&siteStatus=all
Request finished


/Users/gisaac/Documents/GitHub/river_flow_notebooks/scraping_functions.py:112: DtypeWarning: Columns (1,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep="\t")
/Users/gisaac/Documents/GitHub/river_flow_notebooks/scraping_functions.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["datetime"] = df["datetime"].map(lambda x: old_timezone.localize(datetime.strptime(x, "%Y-%m-%d %H:%M")).astimezone(new_timezone))
/Users/gisaac/Documents/GitHub/river_flow_notebooks/scraping_functions.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

Getting request from ASOS
https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station=OWD&data=tmpf&data=dwpf&data=relh&data=feel&data=sknt&data=sped&data=alti&data=mslp&data=drct&data=ice_accretion_1hr&data=p01m&data=vsby&data=gust&data=skyc1&data=peak_wind_gust&data=snowdepth&year1=2023&month1=1&day1=1&year2=2024&month2=2&day2=1&tz=Etc%2FUTC&format=onlycomma&latlon=no&elev=no&missing=M&trace=T&direct=no&report_type=3&report_type=4
      station             valid   tmpf   dwpf   relh   feel  sknt  sped  \
0         OWD  2023-01-01 00:53  56.00  54.00  93.32  55.90  3.00  3.45   
1         OWD  2023-01-01 01:53  56.00  53.00  90.29  55.90  3.00  3.45   
2         OWD  2023-01-01 02:53  56.00  53.00  90.29  55.90  4.00  4.60   
3         OWD  2023-01-01 03:53  55.00  53.00  93.29  55.00  0.00  0.00   
4         OWD  2023-01-01 04:01  56.00  53.00  90.29  55.90  0.00  0.00   
...       ...               ...    ...    ...    ...    ...   ...   ...   
13936     OWD  2024-01-31 19:53 

/Users/gisaac/anaconda3/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [4]:
h.combine_data()

In [5]:
h.joined_df

,hour_updated,p01m,valid,tmpf,dwpf,ice_accretion_1hr,mslp,drct,sped,alti,...,station_id,index,agency_cd,site_no,datetime,tz_cd,cfs,66098_00060_cd,height,66099_00065_cd
0,2023-01-01 05:00:00+00:00,3.30,2023-01-01 04:01,55.750000,52.750000,NaN,1005.5,0.0,0.00,29.71,...,OWD,1,USGS,01104200,2023-01-01 05:00:00+00:00,EST,521.0,A,3.93,A
1,2023-01-01 06:00:00+00:00,1.02,2023-01-01 05:39,56.000000,53.000000,NaN,1004.4,200.0,6.90,29.67,...,OWD,13,USGS,01104200,2023-01-01 06:00:00+00:00,EST,521.0,A,3.93,A
2,2023-01-01 07:00:00+00:00,0.76,2023-01-01 06:06,56.000000,53.000000,NaN,1003.5,210.0,6.90,29.66,...,OWD,25,USGS,01104200,2023-01-01 07:00:00+00:00,EST,515.0,A,3.92,A
3,2023-01-01 08:00:00+00:00,0.00,2023-01-01 07:01,55.333333,52.333333,NaN,1004.1,190.0,3.45,29.63,...,OWD,37,USGS,01104200,2023-01-01 08:00:00+00:00,EST,515.0,A,3.92,A
4,2023-01-01 09:00:00+00:00,0.00,2023-01-01 08:20,55.500000,52.000000,NaN,1003.5,260.0,6.90,29.64,...,OWD,49,USGS,01104200,2023-01-01 09:00:00+00:00,EST,515.0,A,3.92,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9467,2024-01-31 00:00:00+00:00,0.00,2024-01-30 23:53,29.000000,19.000000,NaN,1025.0,70.0,5.75,30.27,...,OWD,113698,USGS,1104200,2024-01-31 00:00:00+00:00,EST,863.0,P,4.47,P
9468,2024-01-31 01:00:00+00:00,0.00,2024-01-31 00:53,29.000000,19.000000,NaN,1024.9,60.0,4.60,30.26,...,OWD,113710,USGS,1104200,2024-01-31 01:00:00+00:00,EST,863.0,P,4.47,P
9469,2024-01-31 02:00:00+00:00,0.00,2024-01-31 01:53,29.000000,19.000000,NaN,1024.5,0.0,0.00,30.25,...,OWD,113722,USGS,1104200,2024-01-31 02:00:00+00:00,EST,856.0,P,4.46,P
9470,2024-01-31 03:00:00+00:00,0.00,2024-01-31 02:53,28.000000,19.000000,NaN,1024.4,0.0,0.00,30.25,...,OWD,113734,USGS,1104200,2024-01-31 03:00:00+00:00,EST,863.0,P,4.47,P


In [6]:
h.combine_snotel_with_df()

KeyError: 'snotel'

## Add Sentinel Images

In [7]:
h.final_df = h.joined_df

In [8]:
import pandas as pd

In [9]:
sentinel_df = pd.read_csv("/Users/gisaac/Downloads/sent_data.csv")

In [10]:
h.combine_sentinel(sentinel_df, data['sentinel_2_tile_id'])
h.final_df

,hour_updated,p01m,valid,tmpf,dwpf,ice_accretion_1hr,mslp,drct,sped,alti,...,site_no,datetime,tz_cd,cfs,66098_00060_cd,height,66099_00065_cd,sensing_time,base_url,SENSING_TIME
0,2023-01-01 05:00:00+00:00,3.30,2023-01-01 04:01,55.750000,52.750000,NaN,1005.5,0.0,0.00,29.71,...,01104200,2023-01-01 05:00:00+00:00,EST,521.0,A,3.93,A,NaN,NaN,NaT
1,2023-01-01 06:00:00+00:00,1.02,2023-01-01 05:39,56.000000,53.000000,NaN,1004.4,200.0,6.90,29.67,...,01104200,2023-01-01 06:00:00+00:00,EST,521.0,A,3.93,A,NaN,NaN,NaT
2,2023-01-01 07:00:00+00:00,0.76,2023-01-01 06:06,56.000000,53.000000,NaN,1003.5,210.0,6.90,29.66,...,01104200,2023-01-01 07:00:00+00:00,EST,515.0,A,3.92,A,NaN,NaN,NaT
3,2023-01-01 08:00:00+00:00,0.00,2023-01-01 07:01,55.333333,52.333333,NaN,1004.1,190.0,3.45,29.63,...,01104200,2023-01-01 08:00:00+00:00,EST,515.0,A,3.92,A,NaN,NaN,NaT
4,2023-01-01 09:00:00+00:00,0.00,2023-01-01 08:20,55.500000,52.000000,NaN,1003.5,260.0,6.90,29.64,...,01104200,2023-01-01 09:00:00+00:00,EST,515.0,A,3.92,A,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9472,2024-01-31 00:00:00+00:00,0.00,2024-01-30 23:53,29.000000,19.000000,NaN,1025.0,70.0,5.75,30.27,...,1104200,2024-01-31 00:00:00+00:00,EST,863.0,P,4.47,P,NaN,NaN,NaT
9473,2024-01-31 01:00:00+00:00,0.00,2024-01-31 00:53,29.000000,19.000000,NaN,1024.9,60.0,4.60,30.26,...,1104200,2024-01-31 01:00:00+00:00,EST,863.0,P,4.47,P,NaN,NaN,NaT
9474,2024-01-31 02:00:00+00:00,0.00,2024-01-31 01:53,29.000000,19.000000,NaN,1024.5,0.0,0.00,30.25,...,1104200,2024-01-31 02:00:00+00:00,EST,856.0,P,4.46,P,NaN,NaN,NaT
9475,2024-01-31 03:00:00+00:00,0.00,2024-01-31 02:53,28.000000,19.000000,NaN,1024.4,0.0,0.00,30.25,...,1104200,2024-01-31 03:00:00+00:00,EST,863.0,P,4.47,P,NaN,NaN,NaT


In [11]:
h.final_df[h.final_df["SENSING_TIME"].notna()]

,hour_updated,p01m,valid,tmpf,dwpf,ice_accretion_1hr,mslp,drct,sped,alti,...,site_no,datetime,tz_cd,cfs,66098_00060_cd,height,66099_00065_cd,sensing_time,base_url,SENSING_TIME
35,2023-01-02 16:00:00+00:00,0.00,2023-01-02 15:53,48.00,33.0,NaN,1018.8,290.0,12.65,30.09,...,01104200,2023-01-02 16:00:00+00:00,EST,481.0,A,3.86,A,2023-01-02 15:51:24.523 UTC,gs://gcp-public-data-sentinel-2/tiles/19/T/CG/...,2023-01-02 16:00:00+00:00
83,2023-01-04 16:00:00+00:00,0.00,2023-01-04 15:06,43.25,40.5,NaN,1013.3,0.0,0.00,29.92,...,01104200,2023-01-04 16:00:00+00:00,EST,481.0,A,3.86,A,2023-01-04 15:41:27.401 UTC,gs://gcp-public-data-sentinel-2/tiles/19/T/CG/...,2023-01-04 16:00:00+00:00
155,2023-01-07 16:00:00+00:00,0.00,2023-01-07 15:53,41.00,34.0,NaN,1018.2,270.0,6.90,30.07,...,01104200,2023-01-07 16:00:00+00:00,EST,458.0,A,3.82,A,2023-01-07 15:51:22.726 UTC,gs://gcp-public-data-sentinel-2/tiles/19/T/CG/...,2023-01-07 16:00:00+00:00
203,2023-01-09 16:00:00+00:00,0.00,2023-01-09 15:53,40.00,28.9,NaN,1012.7,240.0,6.90,29.90,...,01104200,2023-01-09 16:00:00+00:00,EST,437.0,A,3.78,A,2023-01-09 15:41:27.741 UTC,gs://gcp-public-data-sentinel-2/tiles/19/T/CG/...,2023-01-09 16:00:00+00:00
275,2023-01-12 16:00:00+00:00,0.25,2023-01-12 15:53,36.00,30.9,NaN,1025.4,0.0,0.00,30.28,...,01104200,2023-01-12 16:00:00+00:00,EST,353.0,A,3.62,A,2023-01-12 15:51:23.161 UTC,gs://gcp-public-data-sentinel-2/tiles/19/T/CG/...,2023-01-12 16:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8840,2024-01-04 16:00:00+00:00,0.00,2024-01-04 15:53,42.00,30.0,NaN,1012.4,NaN,4.60,29.90,...,1104200,2024-01-04 16:00:00+00:00,EST,653.0,P,4.16,P,2024-01-04 15:41:25.798 UTC,gs://gcp-public-data-sentinel-2/tiles/19/T/CG/...,2024-01-04 16:00:00+00:00
8912,2024-01-07 16:00:00+00:00,3.55,2024-01-07 15:36,37.00,33.0,NaN,1002.3,50.0,13.80,29.61,...,1104200,2024-01-07 16:00:00+00:00,EST,596.0,P,4.07,P,2024-01-07 15:51:21.309 UTC,gs://gcp-public-data-sentinel-2/tiles/19/T/CG/...,2024-01-07 16:00:00+00:00
8960,2024-01-09 16:00:00+00:00,0.00,2024-01-09 15:53,31.00,27.0,NaN,1029.1,0.0,0.00,30.39,...,1104200,2024-01-09 16:00:00+00:00,EST,554.0,P,4.0,P,2024-01-09 15:41:27.813 UTC,gs://gcp-public-data-sentinel-2/tiles/19/T/CG/...,2024-01-09 16:00:00+00:00
9032,2024-01-12 16:00:00+00:00,0.00,2024-01-12 15:53,46.00,28.0,NaN,1022.3,310.0,9.20,30.19,...,1104200,2024-01-12 16:00:00+00:00,EST,928.0,P,4.56,P,2024-01-12 15:51:23.474 UTC,gs://gcp-public-data-sentinel-2/tiles/19/T/CG/...,2024-01-12 16:00:00+00:00


In [13]:
h.write_final_df_to_bq("flow3")

ArrowTypeError: Expected bytes, got a 'int' object

In [ ]:
def combine_sentinel(final_df, sentinel_df, tile):
    """ to combine the Sentinel data with the joined ASOS, USGS, and SNOTEL data.
    """
    sentinel_df = sentinel_df[sentinel_df["mgrs_tile"]==tile] 
    sentinel_df = sentinel_df[["sensing_time", "base_url"]]
    sentinel_df["SENSING_TIME"] = pd.to_datetime(sentinel_df["sensing_time"], utc=True, format='mixed').round('60min')
    final_df =final_df.merge(sentinel_df, left_on="hour_updated", right_on="SENSING_TIME", how="left")
    return final_df

fin_df = combine_sentinel(h.final_df, sentinel_df, "11SMT")
fin_df

In [ ]:
fin_df.dropna(subset=["base_url"])

In [ ]:
 sentinel_df_11 = sentinel_df[sentinel_df["mgrs_tile"]=="11SMT"] 

In [ ]:
sentinel_df_11

In [ ]:
find = fin_df

In [ ]:
find.to_csv("final_11059300.csv")

In [ ]:
data

In [16]:
h.final_df

,hour_updated,p01m,valid,tmpf,dwpf,ice_accretion_1hr,mslp,drct,sped,alti,...,site_no,datetime,tz_cd,cfs,66098_00060_cd,height,66099_00065_cd,sensing_time,base_url,SENSING_TIME
0,2023-01-01 05:00:00+00:00,3.30,2023-01-01 04:01,55.750000,52.750000,NaN,1005.5,0.0,0.00,29.71,...,01104200,2023-01-01 05:00:00+00:00,EST,521.0,A,3.93,A,NaN,NaN,NaT
1,2023-01-01 06:00:00+00:00,1.02,2023-01-01 05:39,56.000000,53.000000,NaN,1004.4,200.0,6.90,29.67,...,01104200,2023-01-01 06:00:00+00:00,EST,521.0,A,3.93,A,NaN,NaN,NaT
2,2023-01-01 07:00:00+00:00,0.76,2023-01-01 06:06,56.000000,53.000000,NaN,1003.5,210.0,6.90,29.66,...,01104200,2023-01-01 07:00:00+00:00,EST,515.0,A,3.92,A,NaN,NaN,NaT
3,2023-01-01 08:00:00+00:00,0.00,2023-01-01 07:01,55.333333,52.333333,NaN,1004.1,190.0,3.45,29.63,...,01104200,2023-01-01 08:00:00+00:00,EST,515.0,A,3.92,A,NaN,NaN,NaT
4,2023-01-01 09:00:00+00:00,0.00,2023-01-01 08:20,55.500000,52.000000,NaN,1003.5,260.0,6.90,29.64,...,01104200,2023-01-01 09:00:00+00:00,EST,515.0,A,3.92,A,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9472,2024-01-31 00:00:00+00:00,0.00,2024-01-30 23:53,29.000000,19.000000,NaN,1025.0,70.0,5.75,30.27,...,1104200,2024-01-31 00:00:00+00:00,EST,863.0,P,4.47,P,NaN,NaN,NaT
9473,2024-01-31 01:00:00+00:00,0.00,2024-01-31 00:53,29.000000,19.000000,NaN,1024.9,60.0,4.60,30.26,...,1104200,2024-01-31 01:00:00+00:00,EST,863.0,P,4.47,P,NaN,NaN,NaT
9474,2024-01-31 02:00:00+00:00,0.00,2024-01-31 01:53,29.000000,19.000000,NaN,1024.5,0.0,0.00,30.25,...,1104200,2024-01-31 02:00:00+00:00,EST,856.0,P,4.46,P,NaN,NaN,NaT
9475,2024-01-31 03:00:00+00:00,0.00,2024-01-31 02:53,28.000000,19.000000,NaN,1024.4,0.0,0.00,30.25,...,1104200,2024-01-31 03:00:00+00:00,EST,863.0,P,4.47,P,NaN,NaN,NaT


In [68]:
def write_final_df_to_bq(h, client, table_name: str) -> bool:
    sample = h.final_df[['hour_updated', 'p01m', 'valid', 'tmpf', 'dwpf', 'ice_accretion_1hr',
       'mslp', 'drct', 'sped', 'alti', 'relh', 'sknt', 'feel', 'vsby', 'gust',
       'skyc1', 'peak_wind_gust', 'snowdepth', 'cfs', 'height']]
    job = client.client.load_table_from_dataframe(sample, table_name)
    print(job.result())
    return True
h.final_df["height"] = pd.to_numeric(h.final_df["height"], errors='coerce')
h.final_df["height"] = pd.to_numeric(h.final_df["height"], errors='coerce')

In [69]:
write_final_df_to_bq(h, h.bq_connect, "hydronet.flow7")

LoadJob<project=hydro-earthnet-db, location=EU, id=13bc0b26-ad9f-45b9-879d-3dfa6cc71066>


True

In [45]:
h.joined_df.columns

Index(['hour_updated', 'p01m', 'valid', 'tmpf', 'dwpf', 'ice_accretion_1hr',
       'mslp', 'drct', 'sped', 'alti', 'relh', 'sknt', 'feel', 'vsby', 'gust',
       'skyc1', 'peak_wind_gust', 'snowdepth', 'station_id', 'index',
       'agency_cd', 'site_no', 'datetime', 'tz_cd', 'cfs', '66098_00060_cd',
       'height', '66099_00065_cd'],
      dtype='object')

In [36]:
h.final_df["index"] = h.final_df["index"].astype('str')

In [35]:
h.final_df["index"]

0            1
1           13
2           25
3           37
4           49
         ...  
9472    113698
9473    113710
9474    113722
9475    113734
9476    113746
Name: index, Length: 9477, dtype: object

In [34]:
write_final_df_to_bq(h, h.bq_connect, "hydronet.flow3")

ArrowTypeError: Expected bytes, got a 'int' object

In [39]:
h.final_df["station_id"] = h.final_df["station_id"].astype('str')

In [41]:
h.final_df.dtypes

hour_updated         datetime64[ns, UTC]
p01m                             float64
valid                             object
tmpf                             float64
dwpf                             float64
ice_accretion_1hr                float64
mslp                             float64
drct                             float64
sped                             float64
alti                             float64
relh                             float64
sknt                             float64
feel                             float64
vsby                             float64
gust                             float64
skyc1                             object
peak_wind_gust                   float64
snowdepth                        float64
station_id                        object
index                             object
agency_cd                         object
site_no                           object
datetime             datetime64[ns, UTC]
tz_cd                             object
cfs             